# Tripod EIT Quantum Memory — Encoder Geometry Analysis

## MSc Thesis Simulation: Diagonal Non-Isometry Framework

This notebook implements a single-atom tripod EIT storage model for analysing **diagonal encoder non-isometry** through population-based Gram matrices.

### Physical Model

- **Atomic system**: $^{87}$Rb D2 line, F=1 ground manifold
- **States**: $|e\rangle$, $|g_{-1}\rangle$, $|g_0\rangle$, $|g_{+1}\rangle$
- **Transitions**:
  - $\sigma^+$ probe: $|g_{-1}\rangle \leftrightarrow |e\rangle$
  - $\pi$ control: $|g_0\rangle \leftrightarrow |e\rangle$
  - $\sigma^-$ probe: $|g_{+1}\rangle \leftrightarrow |e\rangle$

### Key Framework Clarifications

1. **Encoder type**: Classical-field-driven, basis-selective encoder (NOT a linear map on amplitudes)

2. **Gram matrix**: Population-based diagonal matrix $G = \mathrm{diag}(\eta_+, \eta_-)$, measuring basis-state survival probabilities (NOT Hilbert-Schmidt state overlaps)

3. **Fidelity**: Population fidelity only — overlap with target basis state (NOT full quantum state fidelity)

4. **Non-isometry source**: Clebsch-Gordan coefficient asymmetry in spontaneous emission branching

5. **Scope**: Diagonal non-isometry only — no phase-coherent encoding, no DSP interference

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

from qutip import basis, ket2dm, mesolve, Options, Qobj, expect

# Plot settings
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("Tripod EIT Encoder Geometry Analysis")
print("="*45)
print("Framework: Diagonal non-isometry via population-based Gram matrix")
print("Scope: Single-atom, single-channel, basis-selective encoding")

---
## 1. Physical Parameters

In [ ]:
@dataclass
class TripodParameters:
    """
    Physical parameters for tripod EIT quantum memory.
    
    Default values correspond to 87Rb D2 line.
    
    Note on branching ratios:
    - For physical 87Rb F=1 -> F'=0, all CG coefficients are equal (b+=b-=0.5 in effective Lambda)
    - Asymmetric values explore hypothetical/effective asymmetries
    """
    
    # === Atomic Properties ===
    Gamma: float = 2 * np.pi * 6.0666e6  # Spontaneous emission rate [rad/s]
    
    # === Rabi Frequencies ===
    Omega_probe_max: float = 2 * np.pi * 2e6    # Peak probe [rad/s]
    Omega_control_max: float = 2 * np.pi * 30e6  # Peak control [rad/s]
    
    # === Decay Branching Ratios (effective Lambda system) ===
    # For each channel: fraction going to target vs back to g0
    # b_plus + (1-b_plus) = 1 for sigma+ channel
    # b_minus + (1-b_minus) = 1 for sigma- channel
    branch_plus: float = 0.5   # |e> -> |g_{-1}> fraction for sigma+ storage
    branch_minus: float = 0.5  # |e> -> |g_{+1}> fraction for sigma- storage
    
    # === Pulse Timing ===
    t_probe_center: float = 0.5e-6    # Probe pulse center [s]
    tau_probe: float = 0.15e-6        # Probe pulse width [s]
    t_control_ramp: float = 0.7e-6    # Control ramp-down center [s]
    tau_control_ramp: float = 0.2e-6  # Control ramp-down width [s]
    
    # === Ground-State Dephasing ===
    # This is GLOBAL T2 dephasing on all ground states (not just logical)
    # Physical interpretation: magnetic field noise, collisions, etc.
    gamma_dephasing: float = 0.0  # Dephasing rate [rad/s]
    
    # === Simulation ===
    t_start: float = 0.0
    t_end: float = 2.0e-6
    n_steps: int = 2000
    
    @property
    def control_probe_ratio(self):
        """Ratio Omega_c / Omega_p - relevant for adiabaticity."""
        return self.Omega_control_max / self.Omega_probe_max
    
    @property
    def branch_asymmetry(self):
        """Ratio b-/b+ quantifying CG asymmetry."""
        if self.branch_plus > 0:
            return self.branch_minus / self.branch_plus
        return np.inf
    
    def summary(self):
        print("Tripod Parameters")
        print("-" * 40)
        print(f"Γ = {self.Gamma/(2*np.pi*1e6):.2f} MHz")
        print(f"Ω_probe = {self.Omega_probe_max/(2*np.pi*1e6):.1f} MHz")
        print(f"Ω_control = {self.Omega_control_max/(2*np.pi*1e6):.1f} MHz")
        print(f"Control/Probe ratio = {self.control_probe_ratio:.0f}:1")
        print(f"Branch ratios: b+ = {self.branch_plus:.3f}, b- = {self.branch_minus:.3f}")
        print(f"Branch asymmetry b-/b+ = {self.branch_asymmetry:.3f}")
        if self.gamma_dephasing > 0:
            print(f"Global ground-state dephasing: γ_deph/Γ = {self.gamma_dephasing/self.Gamma:.4f}")
        else:
            print("Dephasing: OFF")

# Create default parameters
params = TripodParameters()
params.summary()

---
## 2. Tripod System Definition

In [ ]:
class TripodSystem:
    """
    4-level tripod atomic system for single-channel EIT storage.
    
    Each polarisation channel (sigma+ or sigma-) forms an effective Lambda system:
    - sigma+: |g_{-1}> <-> |e> <-> |g_0>
    - sigma-: |g_{+1}> <-> |e> <-> |g_0>
    
    Spontaneous emission from |e> is restricted to the two states in each
    Lambda subsystem, with branching ratio b determining the split between
    target state and initial state |g_0>.
    
    Dephasing acts on ALL ground states (global T2), not just logical subspace.
    """
    
    def __init__(self, params: TripodParameters):
        self.params = params
        self.dim = 4
        self._setup_basis()
        self._setup_operators()
    
    def _setup_basis(self):
        """Define basis states: |e>, |g_{-1}>, |g_0>, |g_{+1}>."""
        self.e = basis(4, 0)      # Excited state
        self.gm1 = basis(4, 1)    # |g_{-1}>: target for sigma+
        self.g0 = basis(4, 2)     # |g_0>: initial state
        self.gp1 = basis(4, 3)    # |g_{+1}>: target for sigma-
        
        # Projectors
        self.Pe = ket2dm(self.e)
        self.Pgm1 = ket2dm(self.gm1)
        self.Pg0 = ket2dm(self.g0)
        self.Pgp1 = ket2dm(self.gp1)
        
        # Logical subspace projector
        self.P_logical = self.Pgm1 + self.Pgp1
    
    def _setup_operators(self):
        """Define transition operators."""
        self.sigma_e_gm1 = self.e * self.gm1.dag()  # sigma+ transition
        self.sigma_e_g0 = self.e * self.g0.dag()    # pi transition
        self.sigma_e_gp1 = self.e * self.gp1.dag()  # sigma- transition
    
    def get_collapse_ops_sigma_plus(self) -> List[Qobj]:
        """
        Collapse operators for sigma+ storage channel.
        
        Effective Lambda system: |g_{-1}> - |e> - |g_0>
        Decay from |e>:
          - |e> -> |g_{-1}> with rate b+ * Gamma (to target)
          - |e> -> |g_0> with rate (1-b+) * Gamma (recycling)
        
        Global dephasing acts on all ground states.
        """
        p = self.params
        c_ops = [
            np.sqrt(p.branch_plus * p.Gamma) * self.gm1 * self.e.dag(),
            np.sqrt((1 - p.branch_plus) * p.Gamma) * self.g0 * self.e.dag()
        ]
        
        # Global ground-state dephasing (T2 on ALL ground states)
        # This suppresses recycling interference via g0 pathways
        if p.gamma_dephasing > 0:
            c_ops.extend([
                np.sqrt(p.gamma_dephasing) * self.Pgm1,
                np.sqrt(p.gamma_dephasing) * self.Pg0,
                np.sqrt(p.gamma_dephasing) * self.Pgp1
            ])
        
        return c_ops
    
    def get_collapse_ops_sigma_minus(self) -> List[Qobj]:
        """
        Collapse operators for sigma- storage channel.
        
        Effective Lambda system: |g_{+1}> - |e> - |g_0>
        Decay from |e>:
          - |e> -> |g_{+1}> with rate b- * Gamma (to target)
          - |e> -> |g_0> with rate (1-b-) * Gamma (recycling)
        """
        p = self.params
        c_ops = [
            np.sqrt(p.branch_minus * p.Gamma) * self.gp1 * self.e.dag(),
            np.sqrt((1 - p.branch_minus) * p.Gamma) * self.g0 * self.e.dag()
        ]
        
        if p.gamma_dephasing > 0:
            c_ops.extend([
                np.sqrt(p.gamma_dephasing) * self.Pgm1,
                np.sqrt(p.gamma_dephasing) * self.Pg0,
                np.sqrt(p.gamma_dephasing) * self.Pgp1
            ])
        
        return c_ops
    
    def get_initial_state(self) -> Qobj:
        """Initial state: atom in |g_0>."""
        return ket2dm(self.g0)

# Test
system = TripodSystem(params)
print(f"\nSystem dimension: {system.dim}")
print(f"sigma+ collapse operators: {len(system.get_collapse_ops_sigma_plus())}")
print(f"sigma- collapse operators: {len(system.get_collapse_ops_sigma_minus())}")

---
## 3. Time-Dependent Hamiltonian

In [ ]:
class TripodHamiltonian:
    """
    Time-dependent Hamiltonian for single-channel EIT storage.
    
    Physical process (counterintuitive pulse sequence):
    1. Control field ON (strong, creates EIT transparency)
    2. Probe pulse arrives (population excited via dark-state manifold)
    3. Control ramps DOWN (maps excitation into ground-state spin wave)
    
    Note: This does NOT directly transfer |g_0> -> |g_target>.
    Population flows via the excited state and dark-state manifold
    under the counterintuitive driving sequence.
    """
    
    def __init__(self, system: TripodSystem):
        self.system = system
        self.params = system.params
    
    def probe_envelope(self, t: float) -> float:
        """Gaussian probe pulse."""
        p = self.params
        return np.exp(-((t - p.t_probe_center)**2) / (2 * p.tau_probe**2))
    
    def control_envelope(self, t: float) -> float:
        """Control field: starts high, ramps down for storage."""
        p = self.params
        return 0.5 * (1 - np.tanh((t - p.t_control_ramp) / p.tau_control_ramp))
    
    def get_hamiltonian_sigma_plus(self) -> List:
        """
        Hamiltonian for sigma+ storage.
        
        Active transitions:
        - sigma+ probe: |g_{-1}> <-> |e>
        - pi control: |g_0> <-> |e>
        """
        s = self.system
        p = self.params
        
        H_probe = -0.5 * (s.sigma_e_gm1 + s.sigma_e_gm1.dag())
        H_control = -0.5 * (s.sigma_e_g0 + s.sigma_e_g0.dag())
        
        return [
            [H_probe, lambda t, args: p.Omega_probe_max * self.probe_envelope(t)],
            [H_control, lambda t, args: p.Omega_control_max * self.control_envelope(t)]
        ]
    
    def get_hamiltonian_sigma_minus(self) -> List:
        """
        Hamiltonian for sigma- storage.
        
        Active transitions:
        - sigma- probe: |g_{+1}> <-> |e>
        - pi control: |g_0> <-> |e>
        """
        s = self.system
        p = self.params
        
        H_probe = -0.5 * (s.sigma_e_gp1 + s.sigma_e_gp1.dag())
        H_control = -0.5 * (s.sigma_e_g0 + s.sigma_e_g0.dag())
        
        return [
            [H_probe, lambda t, args: p.Omega_probe_max * self.probe_envelope(t)],
            [H_control, lambda t, args: p.Omega_control_max * self.control_envelope(t)]
        ]

In [ ]:
# Visualize pulse sequence
hamiltonian = TripodHamiltonian(system)
tlist = np.linspace(params.t_start, params.t_end, 500)

Omega_p = [params.Omega_probe_max * hamiltonian.probe_envelope(t) / (2*np.pi*1e6) for t in tlist]
Omega_c = [params.Omega_control_max * hamiltonian.control_envelope(t) / (2*np.pi*1e6) for t in tlist]

plt.figure(figsize=(10, 4))
plt.plot(tlist * 1e6, Omega_p, 'b-', label='Probe $\Omega_p$', lw=2)
plt.plot(tlist * 1e6, Omega_c, 'g-', label='Control $\Omega_c$', lw=2)
plt.xlabel('Time (μs)')
plt.ylabel('Rabi frequency (MHz)')
plt.title('EIT Storage Pulse Sequence (Counterintuitive Ordering)')
plt.legend()
plt.tight_layout()
plt.savefig('pulse_sequence.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Encoder Class with Population-Based Gram Matrix

In [ ]:
class TripodEncoder:
    """
    Tripod EIT encoder: extracts and analyses the storage map.
    
    IMPORTANT FRAMEWORK CLARIFICATIONS:
    
    1. This encoder is a MAP ON DENSITY MATRICES, not a linear operator on amplitudes:
       E_pm: rho(0) -> rho_pm
       
    2. The Gram matrix is POPULATION-BASED:
       G = diag(eta_+, eta_-)
       where eta_pm = population transferred to target state
       
       This is NOT the Hilbert-Schmidt Gram matrix Tr(rho_i rho_j).
       The population-based definition isolates efficiency asymmetry
       without conflating it with output-state mixedness.
    
    3. Fidelity here means POPULATION FIDELITY:
       F = <g_target|rho|g_target>
       This is identical to efficiency for pure target states.
       It does NOT test phase coherence or quantum superpositions.
    
    4. Condition number kappa = sqrt(eta_max/eta_min) measures
       DIAGONAL non-isometry only.
    """
    
    def __init__(self, params: TripodParameters):
        self.params = params
        self.system = TripodSystem(params)
        self.hamiltonian = TripodHamiltonian(self.system)
        self.tlist = np.linspace(params.t_start, params.t_end, params.n_steps)
        self.options = Options(rtol=1e-8, atol=1e-8, nsteps=10000)
        
        self.encoder_outputs = {}
        self.dynamics = {}
    
    def run_storage(self, polarization: str) -> Qobj:
        """
        Simulate storage for given polarization channel.
        
        Args:
            polarization: 'sigma_plus' or 'sigma_minus'
            
        Returns:
            Final density matrix
        """
        s = self.system
        h = self.hamiltonian
        rho0 = s.get_initial_state()
        
        if polarization == 'sigma_plus':
            H = h.get_hamiltonian_sigma_plus()
            c_ops = s.get_collapse_ops_sigma_plus()
        else:
            H = h.get_hamiltonian_sigma_minus()
            c_ops = s.get_collapse_ops_sigma_minus()
        
        result = mesolve(H, rho0, self.tlist, c_ops, options=self.options)
        self.dynamics[polarization] = result
        
        return result.states[-1]
    
    def extract_encoder(self, verbose: bool = True) -> Dict[str, Qobj]:
        """Extract encoder by simulating both polarization channels independently."""
        if verbose:
            print("Extracting encoder (single-channel, basis-selective)...")
            print("  Simulating sigma+ channel: |g_0> -> |g_{-1}> via |e>")
        
        self.encoder_outputs['sigma_plus'] = self.run_storage('sigma_plus')
        
        if verbose:
            print("  Simulating sigma- channel: |g_0> -> |g_{+1}> via |e>")
        
        self.encoder_outputs['sigma_minus'] = self.run_storage('sigma_minus')
        
        return self.encoder_outputs
    
    def compute_metrics(self) -> Dict:
        """
        Compute storage metrics for each channel.
        
        Note: 'fidelity' here is population fidelity = efficiency for basis states.
        These are identical quantities in this diagonal framework.
        """
        if not self.encoder_outputs:
            self.extract_encoder(verbose=False)
        
        s = self.system
        metrics = {}
        
        for key, rho in self.encoder_outputs.items():
            # Populations
            pop_e = (s.Pe * rho).tr().real
            pop_gm1 = (s.Pgm1 * rho).tr().real
            pop_g0 = (s.Pg0 * rho).tr().real
            pop_gp1 = (s.Pgp1 * rho).tr().real
            
            # Storage efficiency = population in target state
            if key == 'sigma_plus':
                efficiency = pop_gm1
            else:
                efficiency = pop_gp1
            
            # Population fidelity (= efficiency for basis-state targets)
            # NOT quantum state fidelity - just population overlap
            pop_fidelity = efficiency
            
            # Logical subspace population
            logical_pop = pop_gm1 + pop_gp1
            
            # Purity of output state (for reference, not used in Gram matrix)
            purity = (rho * rho).tr().real
            
            metrics[key] = {
                'efficiency': efficiency,
                'pop_fidelity': pop_fidelity,  # Renamed for clarity
                'logical_pop': logical_pop,
                'leakage_to_g0': pop_g0,
                'excited_residual': pop_e,
                'purity': purity,
                'pop_e': pop_e,
                'pop_gm1': pop_gm1,
                'pop_g0': pop_g0,
                'pop_gp1': pop_gp1
            }
        
        return metrics
    
    def compute_gram_matrix(self) -> Dict:
        """
        Compute POPULATION-BASED encoder Gram matrix and geometry metrics.
        
        IMPORTANT: This is NOT a Hilbert-Schmidt Gram matrix.
        
        G = diag(eta_+, eta_-)
        
        where eta_pm are storage efficiencies (populations in target states).
        
        This definition:
        - Measures basis-state survival probabilities
        - Is diagonal by construction (no coherent cross-talk in single-channel model)
        - Isolates efficiency asymmetry from output-state mixedness
        
        Condition number kappa = sqrt(eta_max/eta_min) quantifies diagonal non-isometry.
        """
        metrics = self.compute_metrics()
        
        eta_plus = metrics['sigma_plus']['efficiency']
        eta_minus = metrics['sigma_minus']['efficiency']
        
        # Population-based Gram matrix (diagonal)
        G = np.array([[eta_plus, 0], [0, eta_minus]])
        
        # Condition number for diagonal non-isometry
        eta_max = max(eta_plus, eta_minus)
        eta_min = min(eta_plus, eta_minus)
        
        if eta_min > 1e-10:
            kappa = np.sqrt(eta_max / eta_min)
        else:
            kappa = np.inf
        
        # Loss-anisotropy decomposition
        eta_avg = (eta_plus + eta_minus) / 2
        
        # Anisotropy (well-defined only when eta_avg > 0)
        if eta_avg > 1e-10:
            anisotropy = abs(eta_plus - eta_minus) / (eta_plus + eta_minus)
        else:
            anisotropy = np.nan  # Undefined in zero-efficiency limit
        
        # Warping from ideal isometry (Frobenius norm deviation)
        G_ideal = eta_avg * np.eye(2)
        warping = np.linalg.norm(G - G_ideal) / (np.linalg.norm(G) + 1e-15)
        
        return {
            'gram_matrix': G,
            'eta_plus': eta_plus,
            'eta_minus': eta_minus,
            'eta_avg': eta_avg,
            'condition_number': kappa,
            'anisotropy': anisotropy,
            'warping': warping,
            'is_isometric': kappa < 1.01  # Numerical tolerance
        }
    
    def print_summary(self):
        """Print comprehensive summary of encoder analysis."""
        metrics = self.compute_metrics()
        gram = self.compute_gram_matrix()
        
        print("\n" + "="*70)
        print("ENCODER GEOMETRY ANALYSIS (Population-Based Diagonal Framework)")
        print("="*70)
        
        print("\n--- Storage Metrics ---")
        for key in ['sigma_plus', 'sigma_minus']:
            m = metrics[key]
            target = '|g_{-1}>' if key == 'sigma_plus' else '|g_{+1}>'
            print(f"\n{key} channel (target: {target}):")
            print(f"  Storage efficiency (eta): {m['efficiency']:.4f}")
            print(f"  Population fidelity: {m['pop_fidelity']:.4f} (= efficiency for basis states)")
            print(f"  Leakage to |g_0>: {m['leakage_to_g0']:.4f}")
            print(f"  Excited residual: {m['excited_residual']:.6f}")
            print(f"  Output purity: {m['purity']:.4f}")
        
        print("\n--- Population-Based Gram Matrix ---")
        print(f"\nG = diag(eta_+, eta_-) = diag({gram['eta_plus']:.4f}, {gram['eta_minus']:.4f})")
        print(f"\n(Note: Off-diagonal = 0 by construction in single-channel model)")
        
        print("\n--- Encoder Geometry ---")
        print(f"Average efficiency (eta_bar): {gram['eta_avg']:.4f}")
        print(f"Condition number (kappa): {gram['condition_number']:.4f}")
        if not np.isnan(gram['anisotropy']):
            print(f"Anisotropy A: {gram['anisotropy']:.4f} (= |eta+ - eta-| / (eta+ + eta-))")
        else:
            print(f"Anisotropy A: undefined (zero efficiency)")
        print(f"Warping from isometry: {gram['warping']*100:.2f}%")
        print(f"\nStatus: {'ISOMETRIC (to numerical precision)' if gram['is_isometric'] else 'NON-ISOMETRIC'}")

---
## 5. Symmetric Case Validation

In [ ]:
# Symmetric branching (should give isometric encoder)
params_sym = TripodParameters()
params_sym.branch_plus = 0.5
params_sym.branch_minus = 0.5

print("SYMMETRIC CASE: b+ = b- = 0.5")
print("(Physical 87Rb F=1 -> F'=0 has symmetric CG coefficients)")
print("="*60)
params_sym.summary()

encoder_sym = TripodEncoder(params_sym)
encoder_sym.extract_encoder()
encoder_sym.print_summary()

In [ ]:
# Plot population dynamics for symmetric case
def plot_dynamics(encoder, title_suffix=""):
    """Plot population dynamics for both channels."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for idx, pol in enumerate(['sigma_plus', 'sigma_minus']):
        ax = axes[idx]
        result = encoder.dynamics[pol]
        s = encoder.system
        t = encoder.tlist * 1e6
        
        pop_e = [expect(s.Pe, st) for st in result.states]
        pop_gm1 = [expect(s.Pgm1, st) for st in result.states]
        pop_g0 = [expect(s.Pg0, st) for st in result.states]
        pop_gp1 = [expect(s.Pgp1, st) for st in result.states]
        
        ax.plot(t, pop_e, 'r-', label=r'$|e\rangle$', lw=2)
        ax.plot(t, pop_gm1, 'b-', label=r'$|g_{-1}\rangle$', lw=2)
        ax.plot(t, pop_g0, 'g-', label=r'$|g_0\rangle$', lw=2)
        ax.plot(t, pop_gp1, 'm-', label=r'$|g_{+1}\rangle$', lw=2)
        
        target = r'$|g_{-1}\rangle$' if pol == 'sigma_plus' else r'$|g_{+1}\rangle$'
        ax.set_xlabel('Time (μs)')
        ax.set_ylabel('Population')
        ax.set_title(f'{pol} channel → {target}')
        ax.legend(loc='right')
        ax.set_ylim([-0.05, 1.05])
    
    plt.suptitle(f'Population Dynamics {title_suffix}', fontsize=14)
    plt.tight_layout()
    return fig

fig = plot_dynamics(encoder_sym, "(Symmetric b+=b-=0.5)")
plt.savefig('dynamics_symmetric.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Asymmetric Case (Non-Isometric Encoder)

In [ ]:
# Asymmetric branching (non-isometric encoder)
params_asym = TripodParameters()
params_asym.branch_plus = 0.5
params_asym.branch_minus = 0.25  # Reduced branching for sigma-

print("ASYMMETRIC CASE: b+ = 0.5, b- = 0.25")
print("(Hypothetical/effective CG asymmetry)")
print("="*60)
params_asym.summary()

encoder_asym = TripodEncoder(params_asym)
encoder_asym.extract_encoder()
encoder_asym.print_summary()

In [ ]:
fig = plot_dynamics(encoder_asym, "(Asymmetric b-/b+ = 0.5)")
plt.savefig('dynamics_asymmetric.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. Branching Ratio Asymmetry Sweep

In [ ]:
def sweep_branching_asymmetry(branch_minus_values: List[float], 
                               branch_plus: float = 0.5,
                               verbose: bool = True) -> Dict:
    """
    Sweep branching ratio asymmetry and analyse encoder geometry.
    
    This tests the core thesis claim: CG asymmetry -> diagonal non-isometry.
    
    Expected scaling (strong asymmetry limit):
    kappa ~ sqrt(b_max / b_min)
    """
    results = {
        'branch_ratio': [],
        'kappa': [],
        'eta_plus': [],
        'eta_minus': [],
        'eta_avg': [],
        'anisotropy': [],
        'warping': []
    }
    
    if verbose:
        print(f"{'b-/b+':<10} {'kappa':<10} {'eta+':<10} {'eta-':<10} {'eta_avg':<10} {'Anisotropy':<10}")
        print("-" * 60)
    
    for b_minus in branch_minus_values:
        ratio = b_minus / branch_plus
        
        p = TripodParameters()
        p.branch_plus = branch_plus
        p.branch_minus = b_minus
        
        enc = TripodEncoder(p)
        enc.extract_encoder(verbose=False)
        gram = enc.compute_gram_matrix()
        
        results['branch_ratio'].append(ratio)
        results['kappa'].append(gram['condition_number'])
        results['eta_plus'].append(gram['eta_plus'])
        results['eta_minus'].append(gram['eta_minus'])
        results['eta_avg'].append(gram['eta_avg'])
        results['anisotropy'].append(gram['anisotropy'])
        results['warping'].append(gram['warping'])
        
        if verbose:
            print(f"{ratio:<10.2f} {gram['condition_number']:<10.4f} "
                  f"{gram['eta_plus']:<10.4f} {gram['eta_minus']:<10.4f} "
                  f"{gram['eta_avg']:<10.4f} {gram['anisotropy']:<10.4f}")
    
    return results

print("\nBranching Ratio Asymmetry Sweep")
print("="*65)
print("Testing: CG asymmetry (b-/b+) -> diagonal non-isometry (kappa)")
print()

branch_minus_values = [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1]
sweep_results = sweep_branching_asymmetry(branch_minus_values)

In [ ]:
# Plot sweep results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Condition number vs asymmetry
ax = axes[0, 0]
ax.plot(sweep_results['branch_ratio'], sweep_results['kappa'], 'bo-', lw=2, ms=8)
ax.axhline(y=1, color='k', ls='--', alpha=0.5, label='Isometric (κ=1)')
ax.set_xlabel(r'Branching ratio $b_-/b_+$', fontsize=12)
ax.set_ylabel(r'Condition number $\kappa$', fontsize=12)
ax.set_title('Diagonal Non-Isometry vs CG Asymmetry', fontsize=14)
ax.legend()

# Efficiencies vs asymmetry
ax = axes[0, 1]
ax.plot(sweep_results['branch_ratio'], sweep_results['eta_plus'], 'b^-', 
         label=r'$\eta_+$ (sigma+)', lw=2, ms=8)
ax.plot(sweep_results['branch_ratio'], sweep_results['eta_minus'], 'rv-', 
         label=r'$\eta_-$ (sigma-)', lw=2, ms=8)
ax.plot(sweep_results['branch_ratio'], sweep_results['eta_avg'], 'g--', 
         label=r'$\bar{\eta}$ (average)', lw=2)
ax.set_xlabel(r'Branching ratio $b_-/b_+$', fontsize=12)
ax.set_ylabel('Storage Efficiency', fontsize=12)
ax.set_title('Polarisation-Dependent Efficiency', fontsize=14)
ax.legend()

# Anisotropy vs asymmetry
ax = axes[1, 0]
ax.plot(sweep_results['branch_ratio'], sweep_results['anisotropy'], 'mo-', lw=2, ms=8)
ax.set_xlabel(r'Branching ratio $b_-/b_+$', fontsize=12)
ax.set_ylabel(r'Anisotropy $A$', fontsize=12)
ax.set_title('Encoder Anisotropy', fontsize=14)

# kappa vs theoretical prediction
ax = axes[1, 1]
kappa_theory = [np.sqrt(0.5 / b) if b > 0 else np.inf for b in branch_minus_values]
ax.plot(sweep_results['branch_ratio'], sweep_results['kappa'], 'bo-', 
         label='Simulated', lw=2, ms=8)
ax.plot(sweep_results['branch_ratio'], kappa_theory, 'r--', 
         label=r'Theory: $\sqrt{b_+/b_-}$', lw=2)
ax.set_xlabel(r'Branching ratio $b_-/b_+$', fontsize=12)
ax.set_ylabel(r'Condition number $\kappa$', fontsize=12)
ax.set_title(r'$\kappa$ Scaling Validation', fontsize=14)
ax.legend()

plt.tight_layout()
plt.savefig('branching_asymmetry_sweep.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved: branching_asymmetry_sweep.png")

---
## 8. Dephasing Effects

**Physical interpretation of dephasing in this model:**

- Dephasing acts on ALL ground states (global T2), not just the logical subspace
- This suppresses recycling interference via |g_0> pathways
- For asymmetric encoders, dephasing can REDUCE anisotropy by randomising phases during storage
- This is NOT an improvement - overall efficiency decreases, but kappa may decrease too

In [ ]:
def sweep_dephasing(gamma_values: List[float],
                    branch_plus: float = 0.5,
                    branch_minus: float = 0.5,
                    verbose: bool = True) -> Dict:
    """
    Sweep ground-state dephasing rate.
    
    Dephasing is GLOBAL T2 on all ground states.
    
    For symmetric encoders: expect kappa ~ 1 throughout (both channels degrade equally)
    For asymmetric encoders: dephasing may reduce kappa by averaging out the asymmetry
    """
    results = {
        'gamma': [],
        'kappa': [],
        'eta_plus': [],
        'eta_minus': [],
        'eta_avg': [],
        'anisotropy': []
    }
    
    if verbose:
        print(f"{'gamma/Gamma':<12} {'kappa':<10} {'eta+':<10} {'eta-':<10} {'eta_avg':<10} {'A':<10}")
        print("-" * 62)
    
    for gamma in gamma_values:
        p = TripodParameters()
        p.branch_plus = branch_plus
        p.branch_minus = branch_minus
        p.gamma_dephasing = gamma * p.Gamma
        
        enc = TripodEncoder(p)
        enc.extract_encoder(verbose=False)
        gram = enc.compute_gram_matrix()
        
        results['gamma'].append(gamma)
        results['kappa'].append(gram['condition_number'])
        results['eta_plus'].append(gram['eta_plus'])
        results['eta_minus'].append(gram['eta_minus'])
        results['eta_avg'].append(gram['eta_avg'])
        results['anisotropy'].append(gram['anisotropy'])
        
        if verbose:
            print(f"{gamma:<12.4f} {gram['condition_number']:<10.4f} "
                  f"{gram['eta_plus']:<10.4f} {gram['eta_minus']:<10.4f} "
                  f"{gram['eta_avg']:<10.4f} {gram['anisotropy']:<10.4f}")
    
    return results

In [ ]:
# Dephasing sweep: SYMMETRIC encoder
print("\nDephasing Sweep: SYMMETRIC encoder (b+ = b- = 0.5)")
print("="*65)
print("Expected: kappa ~ 1 throughout (both channels degrade equally)")
print()

gamma_values = [0.0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1]
deph_sym = sweep_dephasing(gamma_values, branch_plus=0.5, branch_minus=0.5)

In [ ]:
# Dephasing sweep: ASYMMETRIC encoder
print("\nDephasing Sweep: ASYMMETRIC encoder (b+ = 0.5, b- = 0.25)")
print("="*65)
print("Expected: dephasing may REDUCE kappa by averaging out asymmetry")
print("(But overall efficiency decreases)")
print()

deph_asym = sweep_dephasing(gamma_values, branch_plus=0.5, branch_minus=0.25)

In [ ]:
# Plot dephasing results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Symmetric: efficiency vs dephasing
ax = axes[0, 0]
ax.plot(deph_sym['gamma'], deph_sym['eta_plus'], 'b^-', label=r'$\eta_+$', lw=2, ms=8)
ax.plot(deph_sym['gamma'], deph_sym['eta_minus'], 'rv-', label=r'$\eta_-$', lw=2, ms=8)
ax.set_xlabel(r'$\gamma_{deph}/\Gamma$', fontsize=12)
ax.set_ylabel('Storage Efficiency', fontsize=12)
ax.set_title('Symmetric Encoder: Efficiency vs Dephasing', fontsize=14)
ax.legend()

# Symmetric: kappa vs dephasing
ax = axes[0, 1]
ax.plot(deph_sym['gamma'], deph_sym['kappa'], 'go-', lw=2, ms=8)
ax.axhline(y=1, color='k', ls='--', alpha=0.5)
ax.set_xlabel(r'$\gamma_{deph}/\Gamma$', fontsize=12)
ax.set_ylabel(r'Condition number $\kappa$', fontsize=12)
ax.set_title('Symmetric Encoder: Non-Isometry vs Dephasing', fontsize=14)
ax.set_ylim([0.9, 1.5])

# Asymmetric: efficiency vs dephasing
ax = axes[1, 0]
ax.plot(deph_asym['gamma'], deph_asym['eta_plus'], 'b^-', label=r'$\eta_+$', lw=2, ms=8)
ax.plot(deph_asym['gamma'], deph_asym['eta_minus'], 'rv-', label=r'$\eta_-$', lw=2, ms=8)
ax.plot(deph_asym['gamma'], deph_asym['eta_avg'], 'g--', label=r'$\bar{\eta}$', lw=2)
ax.set_xlabel(r'$\gamma_{deph}/\Gamma$', fontsize=12)
ax.set_ylabel('Storage Efficiency', fontsize=12)
ax.set_title('Asymmetric Encoder: Efficiency vs Dephasing', fontsize=14)
ax.legend()

# Asymmetric: kappa and anisotropy vs dephasing
ax = axes[1, 1]
ax.plot(deph_asym['gamma'], deph_asym['kappa'], 'go-', label=r'$\kappa$', lw=2, ms=8)
ax.axhline(y=1, color='k', ls='--', alpha=0.5)
ax.set_xlabel(r'$\gamma_{deph}/\Gamma$', fontsize=12)
ax.set_ylabel(r'Condition number $\kappa$', fontsize=12)
ax.set_title('Asymmetric Encoder: Non-Isometry vs Dephasing', fontsize=14)
ax.legend()

plt.tight_layout()
plt.savefig('dephasing_sweep.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved: dephasing_sweep.png")

---
## 9. Summary Tables for Thesis

In [ ]:
print("\n" + "="*75)
print("THESIS TABLE: Encoder Geometry vs Branching Asymmetry")
print("="*75)
print(f"{'b-/b+':<10} {'eta+':<10} {'eta-':<10} {'eta_avg':<10} {'Anisotropy':<12} {'kappa':<10}")
print("-"*62)
for i in range(len(sweep_results['branch_ratio'])):
    print(f"{sweep_results['branch_ratio'][i]:<10.2f} "
          f"{sweep_results['eta_plus'][i]:<10.4f} "
          f"{sweep_results['eta_minus'][i]:<10.4f} "
          f"{sweep_results['eta_avg'][i]:<10.4f} "
          f"{sweep_results['anisotropy'][i]:<12.4f} "
          f"{sweep_results['kappa'][i]:<10.4f}")

In [ ]:
print("\n" + "="*75)
print("THESIS TABLE: Dephasing Effects on Asymmetric Encoder (b+/b- = 0.5/0.25)")
print("="*75)
print(f"{'gamma/Gamma':<12} {'eta+':<10} {'eta-':<10} {'eta_avg':<10} {'Anisotropy':<12} {'kappa':<10}")
print("-"*64)
for i in range(len(deph_asym['gamma'])):
    print(f"{deph_asym['gamma'][i]:<12.4f} "
          f"{deph_asym['eta_plus'][i]:<10.4f} "
          f"{deph_asym['eta_minus'][i]:<10.4f} "
          f"{deph_asym['eta_avg'][i]:<10.4f} "
          f"{deph_asym['anisotropy'][i]:<12.4f} "
          f"{deph_asym['kappa'][i]:<10.4f}")

---
## 10. Physical Interpretation and Thesis Alignment

### Framework Summary

This notebook implements the **diagonal non-isometry framework** for tripod EIT encoders:

1. **Encoder type**: Classical-field-driven, basis-selective (NOT linear on amplitudes)
2. **Gram matrix**: Population-based diagonal (NOT Hilbert-Schmidt)
3. **Non-isometry metric**: Condition number κ = √(η_max/η_min)
4. **Physical source**: CG asymmetry in spontaneous emission branching

### Key Findings

1. **Symmetric case is isometric**: When b+ = b- (physical 87Rb), κ = 1 to numerical precision

2. **CG asymmetry → diagonal non-isometry**: κ scales as √(b_max/b_min) in strong asymmetry limit

3. **High efficiency ≠ isometry**: An encoder can have high η_avg but large κ (or vice versa)

4. **Dephasing effect on geometry**: 
   - For symmetric encoders: κ ~ 1 preserved (both channels degrade equally)
   - For asymmetric encoders: dephasing may REDUCE κ by averaging out asymmetry
   - This is NOT an improvement - efficiency decreases, but geometry becomes more isometric

### What This Model Does NOT Claim

- No phase-coherent polarisation encoding
- No DSP interference effects
- No full quantum state fidelity (only population fidelity)
- No qubit-level Bloch sphere distortion
- No verification of adiabatic robustness (assumed, not tested)

### Thesis Alignment

This notebook is fully consistent with the thesis framing of:
> "Diagonal non-isometry as the primitive encoder distortion mechanism"

In [ ]:
print("\n" + "="*70)
print("FINAL CONSISTENCY CHECK")
print("="*70)
print("\n✓ Framework: Diagonal non-isometry via population-based Gram matrix")
print("✓ Encoder: Classical-field-driven, basis-selective (not linear on amplitudes)")
print("✓ Gram matrix: G = diag(eta+, eta-) [population-based, NOT Hilbert-Schmidt]")
print("✓ Fidelity: Population fidelity only (= efficiency for basis states)")
print("✓ Dephasing: Global T2 on all ground states (not just logical)")
print("✓ Scope: No phase-coherent encoding, no DSP interference claims")
print("\nThis notebook is aligned with the thesis diagonal non-isometry framework.")